In [1]:
!pip install langdetect

In [2]:
from ibm_watson import ToneAnalyzerV3
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson import ApiException
import json
import os
import pandas as pd
from langdetect import detect
import random
import re
import time
from sklearn.model_selection import train_test_split
import ast

# Raw Data Combination & Preprocessing

In [3]:
path = r'../raw_data/'
files = os.chdir(path)

In [4]:
covid_twitter_data = pd.DataFrame()
data = pd.read_csv('covid2019_0120.csv')
covid_twitter_data = covid_twitter_data.append(data, ignore_index=True)

In [5]:
os.chdir(r'../raw_data')

In [6]:
covid_twitter_data.to_csv('covid2019_0120_0220.csv',index=False)

In [8]:
t = time.process_time()
clean_text = []
non_en = []
error = []
index = 0
for item in covid_twitter_data['text']:
    try:
        index = index +1
        item = re.sub('(http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]|(\xa0(…)*)))+){1}','',item)
        item = re.sub('@[a-zA-Z0-9_.-]+','',item)

        try: 
            if detect(item) == 'en':
                clean_text.append(item)
            else:
                clean_text.append(None)
                non_en.append(index)
        except:
            error.append(index)
            clean_text.append(None)
    except:
        error.append(index)
        clean_text.append(None)

In [38]:
error_df = pd.DataFrame(error)
non_en_df = pd.DataFrame(non_en)
error_df.to_csv('error_df.csv',index=False)
non_en_df.to_csv('non_en_df.csv',index=False)

In [39]:
clean_text = pd.DataFrame(clean_text)
clean_text.columns=['clean_text']
data = pd.concat([covid_twitter_data, clean_text], axis=1)
data.to_csv('covid2019_0120_0220_cleans.csv',index=False)
data.head()

,has_media,hashtags,img_urls,is_replied,is_reply_to,likes,links,parent_tweet_id,replies,reply_to_users,...,text,text_html,timestamp,timestamp_epochs,tweet_id,tweet_url,user_id,username,video_url,clean_text
0,False,[],[],False,True,0,[],1.219387e+18,0,"[{'screen_name': 'JapaDoBonor0', 'user_id': '3...",...,@JapaDoBonor0 o que é esse coronavirus? E se ...,"<p class=""TweetTextSize js-tweet-text tweet-te...",2020-01-20 23:59:57,1579564797,1219409258372046848,/TBorghi/status/1219409258372046848,274278103,Thiago Borghi,NaN,None
1,True,"['coronavirus', 'popalongnowpic']",['https://pbs.twimg.com/media/EOw2PUZXkAANlgu....,False,False,0,[],NaN,0,[],...,I’ve cancelled my delivery! #coronavirus #pop...,"<p class=""TweetTextSize js-tweet-text tweet-te...",2020-01-20 23:59:53,1579564793,1219409242354069504,/exexpatkaren/status/1219409242354069504,40506876,Karen Owen,NaN,I’ve cancelled my delivery! #coronavirus #pop...
2,False,['coronavirus'],[],False,False,0,[],NaN,0,[],...,"Lo último que necesitaba saber, un nuevo virus...","<p class=""TweetTextSize js-tweet-text tweet-te...",2020-01-20 23:59:48,1579564788,1219409220015214592,/JManuelAlamilla/status/1219409220015214592,836304792062406656,JesMa,NaN,None
3,False,[],[],False,False,0,['https://bit.ly/2NJ9DD2'],NaN,0,[],...,"China: el ""coronavirus"" se transmite de person...","<p class=""TweetTextSize js-tweet-text tweet-te...",2020-01-20 23:59:45,1579564785,1219409206954078209,/adbradomin/status/1219409206954078209,1732410342,angel,NaN,None
4,False,[],[],False,False,0,[],NaN,0,[],...,Coronavirus é aquele vírus da Corona cerveja m...,"<p class=""TweetTextSize js-tweet-text tweet-te...",2020-01-20 23:59:42,1579564782,1219409194429960192,/ltsroque/status/1219409194429960192,331665924,The Roque™ 🐻,NaN,None


In [40]:
data_strip_none = data[data.clean_text.notnull()]
data_strip_none = data_strip_none.set_index(pd.Index(range(0,len(data_strip_none))))

In [41]:
data_api, data_test = train_test_split(data_strip_none, train_size=2000, random_state=0)
data_api = data_api.set_index(pd.Index(range(0,len(data_api))))
data_test = data_test.set_index(pd.Index(range(0,len(data_test))))
data_api.to_csv('data_api.csv',index=False)
data_test.to_csv('data_test_0120_0220.csv',index=False)

# IBM Tone Analyzer

In [42]:
data_api = pd.read_csv('data_api.csv')

In [45]:
data_api_12000 = data_api.iloc[0:2000]

In [46]:
data_api_12000.head()

,has_media,hashtags,img_urls,is_replied,is_reply_to,likes,links,parent_tweet_id,replies,reply_to_users,...,text,text_html,timestamp,timestamp_epochs,tweet_id,tweet_url,user_id,username,video_url,clean_text
0,False,['coronavirus'],[],True,False,12,[],NaN,2,[],...,Calmly panic buy all the tamiflu. #coronavirus,"<p class=""TweetTextSize js-tweet-text tweet-te...",2020-01-20 17:21:30,1579540890,1219308984005529602,/barneyfarmer/status/1219308984005529602,235172191,barney farmer,NaN,Calmly panic buy all the tamiflu. #coronavirus
1,False,[],[],False,False,0,['https://www.medscape.com/viewarticle/923944?...,NaN,0,[],...,CDC to Screen for Novel Coronavirus at Three U...,"<p class=""TweetTextSize js-tweet-text tweet-te...",2020-01-20 18:50:13,1579546213,1219331312777035783,/kelvin_munisi/status/1219331312777035783,1201916020212011008,Kelvin Munisi,NaN,CDC to Screen for Novel Coronavirus at Three U...
2,False,[],[],False,False,9,['https://www3.nhk.or.jp/nhkworld/en/news/2020...,NaN,0,[],...,Xi orders stronger steps to contain coronaviru...,"<p class=""TweetTextSize js-tweet-text tweet-te...",2020-01-20 17:30:02,1579541402,1219311134362103809,/NHKWORLD_News/status/1219311134362103809,546657547,NHK WORLD News,NaN,Xi orders stronger steps to contain coronavirus
3,False,[],[],False,False,1,['https://news.sky.com/story/china-confirms-de...,NaN,0,[],...,China confirms deadly Wuhan coronavirus can be...,"<p class=""TweetTextSize js-tweet-text tweet-te...",2020-01-20 18:11:30,1579543890,1219321569908076544,/patrickrooney/status/1219321569908076544,17500537,Patrick Rooney,NaN,China confirms deadly Wuhan coronavirus can be...
4,False,[],[],True,False,0,['https://www.nbcnews.com/news/world/coronavir...,NaN,1,[],...,Coronavirus cases surge in China as virus spre...,"<p class=""TweetTextSize js-tweet-text tweet-te...",2020-01-20 15:17:13,1579533433,1219277710679330816,/Racejude/status/1219277710679330816,152170084,Judy Nagy,NaN,Coronavirus cases surge in China as virus spre...


In [47]:
authenticator = IAMAuthenticator('Fx0GzaRHLRWIz0il2-ihTDVQn862z9faO7gYnP8E9lJb')
tone_analyzer = ToneAnalyzerV3(
    version='2017-09-21',
    authenticator=authenticator
)

tone_analyzer.set_service_url('https://api.us-south.tone-analyzer.watson.cloud.ibm.com/instances/fbbf6fec-d5c1-4d24-be2d-7470a7cc9fdb')
tone_analyzer.set_disable_ssl_verification(False)

In [50]:
tones = []
index = 0
for item in data_api_12000['clean_text']:
    try:
        index = index + 1
        text = item
        tone_analysis = tone_analyzer.tone({'text': text}, content_type='application/json').get_result()
        tones.append(tone_analysis)
    except ApiException as ex:
        tones.append(None)
        print ("Method failed with status code " + str(ex.code) + ": " + ex.message)

In [51]:
tones_copy = tones.copy()
tones_copy = pd.DataFrame(tones_copy)

In [56]:
tones_copy.head()

,document_tone,sentences_tone
0,"{'tones': [{'score': 0.901262, 'tone_id': 'fea...","[{'sentence_id': 0, 'text': 'Calmly panic buy ..."
1,{'tones': []},NaN
2,{'tones': []},NaN
3,"{'tones': [{'score': 0.801827, 'tone_id': 'ana...",NaN
4,{'tones': []},NaN


In [57]:
data_api_12000 = data_api_12000.reset_index()

In [58]:
data_api_12000 = data_api_12000.drop(['index'], axis=1)

In [59]:
data_api_12000 = pd.concat([data_api_12000,tones_copy],axis=1)
data_api_12000.to_csv('data_api_12000.csv',index=False)

# Formating the document tones

In [60]:
data = pd.read_csv('data_api_12000.csv')

In [61]:
data.head()

,has_media,hashtags,img_urls,is_replied,is_reply_to,likes,links,parent_tweet_id,replies,reply_to_users,...,tweet_id,tweet_url,user_id,username,video_url,clean_text,document_tone,sentences_tone,document_tone.1,sentences_tone.1
0,False,['coronavirus'],[],True,False,12,[],NaN,2,[],...,1219308984005529602,/barneyfarmer/status/1219308984005529602,235172191,barney farmer,NaN,Calmly panic buy all the tamiflu. #coronavirus,"{'tones': [{'score': 0.901262, 'tone_id': 'fea...","[{'sentence_id': 0, 'text': 'Calmly panic buy ...","{'tones': [{'score': 0.901262, 'tone_id': 'fea...","[{'sentence_id': 0, 'text': 'Calmly panic buy ..."
1,False,[],[],False,False,0,['https://www.medscape.com/viewarticle/923944?...,NaN,0,[],...,1219331312777035783,/kelvin_munisi/status/1219331312777035783,1201916020212011008,Kelvin Munisi,NaN,CDC to Screen for Novel Coronavirus at Three U...,{'tones': []},NaN,{'tones': []},NaN
2,False,[],[],False,False,9,['https://www3.nhk.or.jp/nhkworld/en/news/2020...,NaN,0,[],...,1219311134362103809,/NHKWORLD_News/status/1219311134362103809,546657547,NHK WORLD News,NaN,Xi orders stronger steps to contain coronavirus,{'tones': []},NaN,{'tones': []},NaN
3,False,[],[],False,False,1,['https://news.sky.com/story/china-confirms-de...,NaN,0,[],...,1219321569908076544,/patrickrooney/status/1219321569908076544,17500537,Patrick Rooney,NaN,China confirms deadly Wuhan coronavirus can be...,"{'tones': [{'score': 0.801827, 'tone_id': 'ana...",NaN,"{'tones': [{'score': 0.801827, 'tone_id': 'ana...",NaN
4,False,[],[],True,False,0,['https://www.nbcnews.com/news/world/coronavir...,NaN,1,[],...,1219277710679330816,/Racejude/status/1219277710679330816,152170084,Judy Nagy,NaN,Coronavirus cases surge in China as virus spre...,{'tones': []},NaN,{'tones': []},NaN


In [62]:
data = data.iloc[:,1:26]

In [63]:
data['document_tones'] = data.document_tone.apply(lambda x: ast.literal_eval(x)['tones'])
data.drop('document_tone',axis = 1,inplace = True)

In [64]:
for i in range(0, len(data)):
    for j in range(0, len(data.loc[i, 'document_tones'])):
        dic = data.loc[i, 'document_tones'][j]
        source = dic['tone_id']
        data.loc[i, source] = 1
data.head()

,hashtags,img_urls,is_replied,is_reply_to,likes,links,parent_tweet_id,replies,reply_to_users,retweets,...,document_tone.1,sentences_tone.1,document_tones,fear,confident,analytical,sadness,joy,tentative,anger
0,['coronavirus'],[],True,False,12,[],NaN,2,[],1,...,"{'tones': [{'score': 0.901262, 'tone_id': 'fea...","[{'sentence_id': 0, 'text': 'Calmly panic buy ...","[{'score': 0.901262, 'tone_id': 'fear', 'tone_...",1.0,1.0,NaN,NaN,NaN,NaN,NaN
1,[],[],False,False,0,['https://www.medscape.com/viewarticle/923944?...,NaN,0,[],0,...,{'tones': []},NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,[],[],False,False,9,['https://www3.nhk.or.jp/nhkworld/en/news/2020...,NaN,0,[],3,...,{'tones': []},NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,[],[],False,False,1,['https://news.sky.com/story/china-confirms-de...,NaN,0,[],1,...,"{'tones': [{'score': 0.801827, 'tone_id': 'ana...",NaN,"[{'score': 0.801827, 'tone_id': 'analytical', ...",NaN,NaN,1.0,NaN,NaN,NaN,NaN
4,[],[],True,False,0,['https://www.nbcnews.com/news/world/coronavir...,NaN,1,[],0,...,{'tones': []},NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [65]:
data.to_csv('sentiment_12000.csv',index=False)

# Combining all datasets with tones analyzed by IBM

In [68]:
data

,hashtags,img_urls,is_replied,is_reply_to,likes,links,parent_tweet_id,replies,reply_to_users,retweets,...,document_tone.1,sentences_tone.1,document_tones,fear,confident,analytical,sadness,joy,tentative,anger
0,['coronavirus'],[],True,False,12,[],NaN,2,[],1,...,"{'tones': [{'score': 0.901262, 'tone_id': 'fea...","[{'sentence_id': 0, 'text': 'Calmly panic buy ...","[{'score': 0.901262, 'tone_id': 'fear', 'tone_...",1.0,1.0,NaN,NaN,NaN,NaN,NaN
1,[],[],False,False,0,['https://www.medscape.com/viewarticle/923944?...,NaN,0,[],0,...,{'tones': []},NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,[],[],False,False,9,['https://www3.nhk.or.jp/nhkworld/en/news/2020...,NaN,0,[],3,...,{'tones': []},NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,[],[],False,False,1,['https://news.sky.com/story/china-confirms-de...,NaN,0,[],1,...,"{'tones': [{'score': 0.801827, 'tone_id': 'ana...",NaN,"[{'score': 0.801827, 'tone_id': 'analytical', ...",NaN,NaN,1.0,NaN,NaN,NaN,NaN
4,[],[],True,False,0,['https://www.nbcnews.com/news/world/coronavir...,NaN,1,[],0,...,{'tones': []},NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,[],[],False,False,0,['https://www.gulf-insider.com/coronavirus-spr...,NaN,0,[],0,...,"{'tones': [{'score': 0.542428, 'tone_id': 'joy...",NaN,"[{'score': 0.542428, 'tone_id': 'joy', 'tone_n...",NaN,NaN,NaN,NaN,1.0,NaN,NaN
1996,[],[],False,False,0,['https://www.stlucianewsonline.com/human-to-h...,NaN,0,[],0,...,"{'tones': [{'score': 0.955445, 'tone_id': 'ana...",NaN,"[{'score': 0.955445, 'tone_id': 'analytical', ...",NaN,NaN,1.0,NaN,NaN,NaN,NaN
1997,[],[],False,False,0,[],NaN,0,[],0,...,"{'tones': [{'score': 0.665589, 'tone_id': 'fea...","[{'sentence_id': 0, 'text': 'Gotta love how th...","[{'score': 0.665589, 'tone_id': 'fear', 'tone_...",1.0,1.0,1.0,NaN,NaN,NaN,NaN
1998,['coronavirus'],[],False,False,1,[],NaN,0,[],0,...,{'tones': []},NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [177]:
pd.read_csv(str(path) + 'sentiment_7000.csv',encoding='cp1252')

,has_media,hashtags,img_urls,is_replied,is_reply_to,likes,links,parent_tweet_id,replies,reply_to_users,...,clean_text,sentences_tone,document_tones,tentative,analytical,confident,sadness,anger,joy,fear
0,False,"['Covid19', 'coronavirus', 'CoronavirusOutbreak']",[],True,True,4,[],1.230000e+18,1,"[{'screen_name': 'RecordGuyBln', 'user_id': '1...",...,"The infection rate of #Covid19 is so high, 26%...","[{'sentence_id': 0, 'text': 'The infection rat...","[{'score': 0.820919, 'tone_id': 'tentative', '...",1.0,NaN,NaN,NaN,NaN,NaN,NaN
1,False,[],[],False,False,0,['https://www.hawaiinewsnow.com/2020/02/14/gov...,NaN,0,[],...,State: Japanese visitor recently in Hawaii con...,NaN,"[{'score': 0.920855, 'tone_id': 'analytical', ...",NaN,1.0,NaN,NaN,NaN,NaN,NaN
2,False,[],[],False,False,0,['https://flip.it/Rt9yhL'],NaN,0,[],...,California lab says it discovered coronavirus ...,NaN,"[{'score': 0.882284, 'tone_id': 'analytical', ...",NaN,1.0,NaN,NaN,NaN,NaN,NaN
3,False,[],[],True,False,45,['https://twitter.com/JHKMMA/status/1216754396...,NaN,3,[],...,"Update! According to my sources, Contender Ser...","[{'sentence_id': 0, 'text': 'Update!', 'tones'...","[{'score': 0.952514, 'tone_id': 'analytical', ...",NaN,1.0,1.0,NaN,NaN,NaN,NaN
4,False,[],[],True,False,2,[],NaN,1,[],...,i went to seattle for svt and there was someon...,NaN,"[{'score': 0.708604, 'tone_id': 'sadness', 'to...",1.0,NaN,NaN,1.0,NaN,NaN,NaN
5,False,[],[],False,False,1,[],NaN,0,[],...,Spokane Regional Health Officer Dr. Bob Lutz s...,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,False,[],[],True,True,5,[],1.230000e+18,2,"[{'screen_name': 'LynAldenContact', 'user_id':...",...,Many countries exporting to China are not runn...,"[{'sentence_id': 0, 'text': 'Many countries ex...","[{'score': 0.50505, 'tone_id': 'sadness', 'ton...",NaN,1.0,NaN,1.0,NaN,NaN,NaN
7,False,[],[],False,False,0,['https://www.forbes.com/sites/lisettevoytko/2...,NaN,0,[],...,Trump Proposes 16% Cut To CDC As Global Number...,NaN,"[{'score': 0.571834, 'tone_id': 'sadness', 'to...",NaN,1.0,NaN,1.0,NaN,NaN,NaN
8,False,"['Lunar', 'NewYear', 'HongKong', 'Wuhan']",[],False,False,0,['https://www.nytimes.com/2020/01/25/world/asi...,NaN,0,[],...,#Lunar #NewYear celebrations are being cancele...,"[{'sentence_id': 0, 'text': '#Lunar #NewYear c...","[{'score': 0.640331, 'tone_id': 'analytical', ...",NaN,1.0,NaN,NaN,NaN,NaN,NaN
9,False,"['HONGKONG', 'WuhanCoronavius', 'CORONAVIRUS']",[],True,False,0,[],NaN,1,[],...,And still as supermarket opened THERE ARE NO T...,"[{'sentence_id': 0, 'text': 'And still as supe...","[{'score': 0.786583, 'tone_id': 'anger', 'tone...",NaN,NaN,NaN,NaN,1.0,NaN,NaN


In [9]:
path = r'../raw_data/'
files = os.chdir(path)
covid_twitter_data = pd.DataFrame()
data = pd.read_csv('sentiment_12000.csv')
covid_twitter_data = covid_twitter_data.append(data, ignore_index=True)

In [10]:
covid_twitter_data.to_csv('sentiment12000full.csv',index=False)

In [34]:
k = 100

In [35]:
print('Tweet : {}\n'.format(covid_twitter_data['text'][k]))
print('Fear = {}\nAnger = {}\nSadness = {}\nConfident = {}\nAnalytical = {}\nJoy = {}'
.format(covid_twitter_data['fear'][k],covid_twitter_data['anger'][k],covid_twitter_data['sadness'][k],covid_twitter_data['confident'][k],covid_twitter_data['analytical'][k],covid_twitter_data['joy'][k]))

Tweet : I hope this coronavirus takes me out. All aboard the S.S. Depression!

Fear = nan
Anger = nan
Sadness = 1.0
Confident = nan
Analytical = nan
Joy = nan


In [49]:
(275/2000)*100  joy

13.750000000000002

In [50]:
(900/2000)*100  analytical

45.0

In [51]:
(170/2000)*100  confident

8.5

In [52]:
(150/2000)*100  sadness

7.5

In [53]:
(32/2000)*100  anger

1.6

In [54]:
(141/2000)*100  fear

7.049999999999999